# Imports

In [94]:
import math

# A

## Initialization

In [119]:
a = 0
b = 3
interval = [a,b]
Fs = [1,1]
EPSILON = 1
length = (interval[1]-interval[0])/EPSILON
while Fs[-1] < length:
    Fs.append(Fs[-1] + Fs[-2])
Fs.pop()

3

## setting Initial points

In [120]:
k = len(Fs)

x1 = a + (Fs[k-3] / Fs[k-1]) * (b - a)
x2 = a + (Fs[k-2] / Fs[k-1]) * (b - a)

ITERATIONS = k - 2

## Computing x1 and x2 in F(X)

In [121]:
for i in range(ITERATIONS):
    newInterval = []
    f_X1 = 2 * (x1)**2 - 5 * x1 + 3
    f_X2 = 2 * (x2)**2 - 5 * x2 + 3
    if f_X1 > f_X2:
        a = x1
        x1 = x2 
        x2 = a+ (Fs[k-2]/Fs[k-1]) * (b-a)
    else:
        b = x2
        x2 = x1
        x1 = a + (Fs[k-3]/Fs[k-1]) * (b-a)
    newInterval.append(a)
    newInterval.append(b)
    print(newInterval)
    k-=1

[0, 1.5]


## Results

In [122]:
finalInterval = []
finalInterval.append(a)
finalInterval.append(b)
print(finalInterval)
minimum = (a+b) / 2
print(minimum)
f_min = 2 * (minimum)**2 - 5 * minimum + 3
print(f_min)

[0, 1.5]
0.75
0.375


# B

In [112]:
a = 0
b = 3
interval = [a,b]
EPSILON = 0.01
GOLDENRATIO = (math.sqrt(5)-1) / 2

## Initialize interior Points

In [113]:
x1 = a + (1-GOLDENRATIO) * (b - a)
x2 = a + GOLDENRATIO * (b - a)

## Substitute in function

In [114]:
f_X1 = 2 * (x1)**2 - 5 * x1 + 3
f_X2 = 2 * (x2)**2 - 5 * x2 + 3
while (b - a) > EPSILON:
    currentInterval = []
    if f_X1  < f_X2:
        b = x2
        x2 = x1
        x1 = a + (1-GOLDENRATIO) * (b-a)
    elif f_X1 > f_X2:
        a = x1
        x1 = x2
        x2 = a + GOLDENRATIO * (b-a)
    f_X1 = 2 * (x1)**2 - 5 * x1 + 3
    f_X2 = 2 * (x2)**2 - 5 * x2 + 3
    currentInterval.append(a)
    currentInterval.append(b)
    print(currentInterval)
minimum = (a+b) / 2
print(minimum)
f_min = 2 * (minimum)**2 - 5 * minimum + 3
print(f_min)

[0, 1.8541019662496847]
[0.7082039324993691, 1.8541019662496847]
[0.7082039324993691, 1.4164078649987384]
[0.9787137637477918, 1.4164078649987384]
[1.1458980337503153, 1.4164078649987384]
[1.1458980337503153, 1.3130823037528392]
[1.20975674250694, 1.3130823037528392]
[1.20975674250694, 1.2736154512635647]
[1.2341485987742902, 1.2736154512635647]
[1.2341485987742902, 1.2585404550416401]
[1.2434654588197158, 1.2585404550416401]
[1.2434654588197158, 1.2527823188651412]
1.2481238888424286
-0.12499296041384866


# C